In [15]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [17]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [18]:
vocab_size = len(char_to_idx)

In [22]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
# import impl.NN as nn
import impl.RNN as rnn

class GRU(rnn.RNN):

    def __init__(self, D, H, char2idx, idx2char):
        super().__init__(D, H, char2idx, idx2char)

    def forward(self, X, h_old, train=True):
        m = self.model
        Wz, Wh, Wy = m['Wz'], m['Wh'], m['Wy']
        bz, bh, by = m['bz'], m['bh'], m['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        X = np.column_stack((h_old, X_one_hot))
        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        X_prime = np.column_stack((h_old, X_one_hot))
        hh, hh_cache = l.fc_forward(X_prime, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        h = (1. - hz) * h_old + hz * hh
        y, y_cache = l.fc_forward(h, Wy, by)

        cache = (
            X, X_prime, h_old, hz, hz_cache, hz_sigm_cache,
            hh, hh_cache, hh_tanh_cache, h, y_cache
        )

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, y_pred, y_train, dh_next, cache):
        X, X_prime, h_old, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache

        dy = loss_fun.dcross_entropy(y_pred, y_train)

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next

        dhh = hz * dh
        dh_old1 = (1. - hz) * dh
        dhz = hh * dh - h_old * dh

        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX_prime, dWh, dbh = l.fc_backward(dhh, hh_cache)
        dh_old2 = dX_prime[:, :self.H]

        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dX, dWz, dbz = l.fc_backward(dhz, hz_cache)
        dh_old3 = dX[:, :self.H]

        dh_next = dh_old1 + dh_old2 + dh_old3

        grad = dict(Wz=dWz, Wh=dWh, Wy=dWy, bz=dbz, bh=dbh, by=dby)

        return grad, dh_next

    def _init_model(self, D, C, H):
        Z = H + D

        self.model = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(D / 2.),
            bz=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [ ]:
time_step = 10
n_iter = 100000 * 2 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [23]:
# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [24]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3754
 hast txr 1tirtpand taueumof is th scch lan Jadat iand mwi It  an UNye 3cCort ite Soluu id bthinii 1


Iter-2000 loss: 2.7247
o ncingth mhend in the itrcy lecs uh. Irdean ugopecIougy wy 7n th8. Thed buliag an ceri. the 9mipetu


Iter-3000 loss: 2.3030
rtolle7, fareenisen Turd-,iG G2o ana cinae band J8e5test nikg rs Japan late eo lo ul toukh an 1f" in


Iter-4000 loss: 2.0090
uest Cmandctedstily irg in 1940, A75, atuon lofest enjise TokhesW puny Narentid sisthe the the gng U


Iter-5000 loss: 1.7865
ctotal e3th inllaved of che Eoppan efe ramio Chonto lawer Canfox of Jatanositf. forthe G8rbaE Holita


Iter-6000 loss: 1.6127
oinaly. AAlacomes iod oret is thites ihes intt crearz5y SI, it on en:(94g and ome wat redionked eanj


Iter-7000 loss: 1.4486
a an ieat-Japan la giold's toet Sinis the popol city of 6wo llloy tue Ondenuped pinceatitO asers ofl


Iter-8000 loss: 1.3178
in tSon exth. walled 2ly. Irtor flase pante Warg cn and Khe trat hichithe Gsurth ofirga ind pou

Iter-25000 loss: 0.6021
r militaris bat ofestharanter and Toud cofint ough ts. TaicoChoshitary the world's topth fighiral er


Iter-26000 loss: 0.5783
 precliry sixth periund pountry but9d Suves penceuied of 186, wokyo hagtent. meside of aioods lere p


Iter-27000 loss: 0.5742
hish atithe worita1 me parlite the nal the Upprrex. Japan was revelagenth garal ofperco end linese l


Iter-28000 loss: 0.5525
volgatal eseapand cant I argas in constitution is the Sta e tonstaleta t  te the G7, the wothe histh


Iter-29000 loss: 0.5285
ipronsmally two d sinh in the our thireld and into and Japontectareat rinonthin Alimonthichest rages


Iter-30000 loss: 0.5151
ffingwas maun open pe ia 19th and Iied ove folloCCy perilytory , tho touxth tho warag twe gevolowes 


Iter-31000 loss: 0.4996
aned oreas late cion "Ssa, four C8 ansen ly mintich wother ard ofer Index whottate regnomeorio oist 


Iter-32000 loss: 0.4801
kkkid to es the fightha Ampor o, by the Global Locflatirnte merity tre the surity, hagh

Iter-49000 loss: 0.3259
 and bith in the Gl of in the world's largest echithit countrd in the naith larly is thig the world'


Iter-50000 loss: 0.3044
st regldomenmiinty in the world's lerctGriropFint Succans it the world's langertholargest if toron t


Iter-51000 loss: 0.2973
e the world's fount Imprried pointainty regionst im7 Ch the world's tenth in the Global Peace Indere


Iter-52000 loss: 0.2986
ty of Japan, whith is the sural, wh cenfion 1945 from the 1st the Sec cons wost as ithes in thintrem


Iter-53000 loss: 0.2927
anes. The First ofinanime lort ncabinato-Japan is reskerofW, is in the First country in Asia const r


Iter-54000 loss: 0.2900
my by porits in the dividectures, ind World War Iuder and ited into exparted llopenter the UNina inc


Iter-55000 loss: 0.2812
wro, strets raic the worlO, was ragicans whe four ", and it the world's liniding prefecons the UIpin


Iter-56000 loss: 0.2881
e Emperor. Japan was whon military byon I areas make wiono Apporex, ato the UN, the Sum

Iter-73000 loss: 0.2428
anji that make colest in the Churox was ma copelicic the Paliencinate 1937 of 1bol to the East China


Iter-74000 loss: 0.2046
s. Japanese Woriagly amed be sowitha amithy sexpovilake piraide War, whiran is an ede toran Diet. Ja


Iter-75000 loss: 0.2481
ta a ur 9.15 the G8, and the G20 has ofor life ex. Japandic and batod Tokyo, the capital wopin lacas


Iter-76000 loss: 0.2137
 The firnol eagl of the Empere wory alyband upercalled in the Chintsouthe woth cama uped an eteop fo


Iter-77000 loss: 0.1973
ed the "Land of the Russo-Japan in the cornts rarc the Global comben the nctyonf utprogdic Gamathita


Iter-78000 loss: 0.2288
and u, Hich shouran in the G8, and the First, pewateleald ofinony. TouepentKy and the G20 and is con


Iter-79000 loss: 0.2065
ond Sino-Japanese War c an iactud States floeleol China Sea, China, Korea and Japan was the first in


Iter-80000 loss: 0.1842
he Seast in the OECD and the fourth leading global city in the world's largest metropol

Iter-97000 loss: 0.1823
ke uppred of Japan's total fountry wits amperd of the Rusto-Japan's hastorys the world's end militar


Iter-98000 loss: 0.1934
he Global Conshifents and the world's eighth larges ams eraned as ehe hich of Japan in il Chinaineas


Iter-99000 loss: 0.1890
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 


Iter-100000 loss: 0.1673
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch




In [ ]:
=========================================================================
Iter-99000 loss: 0.1890
=========================================================================
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 
=========================================================================


=========================================================================
Iter-100000 loss: 0.1673
=========================================================================
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch
=========================================================================


Out[24]:
<__main__.GRU at 0x10d1f7b00>

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>